### Packages

In [ ]:
## packages
import math
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.initializers import GlorotUniform, GlorotNormal, HeUniform, HeNormal, Zeros, Ones
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import load_model
from scipy.optimize import minimize
from scipy.stats import linregress
from joblib import Parallel, delayed
import psutil

### Real-Time Data Processing

In [ ]:
## set of functions used to read and process the raw spectra
def read_files(loc):
    files_all = sorted(os.listdir(loc), key = len)
    ABS_names = []
    PL_names = []
    ABS_all = []
    PL_all = []
    for file in files_all:
        if (file.startswith("Abs")):
            ABS = pd.read_csv(loc + "//" + file , header = None).to_numpy()
            ABS_all.append(ABS)
            ABS_names.append(file.split(".")[0])
        elif (file.startswith("PL")):
            PL = pd.read_csv(loc + "//" + file, header = None).to_numpy()
            PL_all.append(PL)
            PL_names.append(file.split(".")[0])
        elif(file.startswith("Wavelength_Abs")):
            WL_ABS = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("Wavelength_PL")):
            WL_PL = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("DR_Abs")):
            DR_ABS = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("DR_PL")):
            DR_PL = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("LR")):
            LR = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("FR")):
            FR = pd.read_csv(loc + "//" + file, header = None).to_numpy() # FR is the file including experimental conditions 
    return WL_ABS, WL_PL, DR_ABS, DR_PL, LR, ABS_all, PL_all, ABS_names, PL_names, FR
def idx_min(y, x):
    diff = np.abs(y[:, 0] - x)
    idx = diff.argmin()
    return idx
# extract reactive phase
def extract(x, idx1, idx2, numFirstElements):
    x_mean = x[idx1:idx2 + 1, :].mean(axis = 0)
    x_sort = np.sort(x_mean)
    idx_phase = np.nonzero(np.in1d(x_mean, x_sort[:numFirstElements]))[0]
    x_phase = x[:, idx_phase]
    return x_phase
def spectra_avg(x):
    x_avg = x.mean(axis = 1) # avg over extracted reactive phase spectra for each WL; will result in a row matrix
    x_avg = x_avg[:, np.newaxis]
    return x_avg
def baseline_zero(x, idx_low, idx_high):
    x_baseline_zero = x - x[idx_low:idx_high + 1, :].mean() # make baseline zero; subtracting mean of spectra at LL - HL nm
    return x_baseline_zero
def linear_int_x(y, x, i, y_btw):
    x_btw = x[i, 0] - ((x[i + 1, 0] - x[i, 0]) / (y[i + 1, 0] - y[i, 0])) * (y[i, 0] - y_btw)
    return x_btw
def linear_int_y(y, x, i, x_btw):
    y_btw = y[i, 0] - ((y[i + 1, 0] - y[i, 0]) / (x[i + 1, 0] - x[i, 0])) * (x[i, 0] - x_btw)
    return y_btw
def peak_info(y, x):
    area_peak = np.trapz(y.T, x = x.T)[0] # emission peak area
    return area_peak
# process ABS
def spectra_extract_ABS(WL, DR, LR, ABS_all, ABS_names):
    # baseline WL limits
    baseline_LL = 550
    baseline_HL = 600
    idx_WL_LL = idx_min(WL, baseline_LL)
    idx_WL_HL = idx_min(WL, baseline_HL)
    # to extract reactive phase
    lastPeak_LL = 310
    lastPeak_HL = 340 
    idx_lastPeak_LL = idx_min(WL, lastPeak_LL)
    idx_lastPeak_HL = idx_min(WL, lastPeak_HL)
    # excitation WL info
    excitation_WL = 365
    idx_excitationWL = idx_min(WL, excitation_WL)
    # initiate files to be exported
    WLABS_processed = WL
    ABS_excitationWL_list = []
    # ABS_excitationWL_list = np.array([["Absorbance at excitation WL"]])
    for (item, name) in zip(ABS_all, ABS_names):
        ABS_only = item[1:, :]
        ## extract reactive phase and remove carrier phase (PFO)
        param = 100 # number to average
        ABS_phase = extract(ABS_only, idx_lastPeak_LL, idx_lastPeak_HL, param)
        ABS_phase_avg = spectra_avg(ABS_phase) 
        arg_log = (ABS_phase_avg - DR) / (LR - DR) # Beer-Lambert law
        ABS_processed = - np.log10(arg_log) # Beer-Lambert law
        ABS_processed_baseline = baseline_zero(ABS_processed, idx_WL_LL, idx_WL_HL) 
        WLABS_processed = np.concatenate( [WLABS_processed, ABS_processed_baseline] , axis = 1) # attach WL and processed ABS
        # WLABS_processed_filtered = WLABS_processed[np.isnan(WLABS_processed[:, 1]) == False] # remove NaN ABS
        ABS_excitationWL = linear_int_y(ABS_processed_baseline, WL, idx_excitationWL, excitation_WL)
        ABS_excitationWL_list.append(ABS_excitationWL)
        # ABS_excitationWL_list = np.concatenate( [ABS_excitationWL_list, np.array([ABS_excitationWL])[:, np.newaxis]] , axis = 1)
    ABS_excitationWL_list = np.array(ABS_excitationWL_list)[:, np.newaxis]
    return ABS_excitationWL_list
# process PL
def spectra_extract_PL(WL, DR, PL_all, PL_names):
    ## baseline WL limits
    baseline_LL = 1050
    baseline_HL = 1100
    idx_WL_LL = idx_min(WL, baseline_LL)
    idx_WL_HL = idx_min(WL, baseline_HL)
    # to extract reactive phase
    excWL_LL = 315
    excWL_HL = 415
    idx_excWL_LL = idx_min(WL, excWL_LL)
    idx_excWL_HL = idx_min(WL, excWL_HL)
    # emission peak 1 WL limits
    emPeak1_LL = 395 
    emPeak1_HL = 450
    idx_emPeak1_LL = idx_min(WL, emPeak1_LL)
    idx_emPeak1_HL = idx_min(WL, emPeak1_HL)
    WL_aroundPeak1 = WL[idx_emPeak1_LL: idx_emPeak1_HL + 1, :]
    # emission peak 2 WL limits
    emPeak2_LL = 550
    emPeak2_HL = 800
    idx_emPeak2_LL = idx_min(WL, emPeak2_LL)
    idx_emPeak2_HL = idx_min(WL, emPeak2_HL)
    WL_aroundPeak2 = WL[idx_emPeak2_LL: idx_emPeak2_HL + 1, :]
    # emission peak 3 WL limits
    emPeak3_LL = 900 
    emPeak3_HL = 1100
    idx_emPeak3_LL = idx_min(WL, emPeak3_LL)
    idx_emPeak3_HL = idx_min(WL, emPeak3_HL)
    WL_aroundPeak3 = WL[idx_emPeak3_LL: idx_emPeak3_HL + 1, :]
    ## initiate files to be exported
    WLPL_processed = WL
    emission_areas = []
    # emission_details = np.array([["Emission Peak WL"], ["Emission Peak Intensity"], ["FWHM"], ["Emission Peak Area"]])
    for (item, name) in zip(PL_all, PL_names):
        PL_only = item[1:, :]
        ## extract reactive phase and remove carrier phase (PFO)
        param = 50
        PL_phase = extract(PL_only, idx_excWL_LL, idx_excWL_HL, param) 
        PL_phase_avg = spectra_avg(PL_phase)
        PL_processed = PL_phase_avg - DR
        PL_processed_baseline = baseline_zero(PL_processed, idx_WL_LL, idx_WL_HL) 
        WLPL_processed = np.concatenate( [WLPL_processed, PL_processed_baseline] , axis = 1) # attach WL and processed PL
        ## emission peak 1 area
        PL_aroundPeak1 = PL_processed_baseline[idx_emPeak1_LL: idx_emPeak1_HL + 1, :]
        area_peak1 = peak_info(PL_aroundPeak1, WL_aroundPeak1)
        ## emission peak 2 area
        PL_aroundPeak2 = PL_processed_baseline[idx_emPeak2_LL: idx_emPeak2_HL + 1, :]
        area_peak2 = peak_info(PL_aroundPeak2, WL_aroundPeak2)
        ## emission peak 3 area
        PL_aroundPeak3 = PL_processed_baseline[idx_emPeak3_LL: idx_emPeak3_HL + 1, :]
        area_peak3 = peak_info(PL_aroundPeak3, WL_aroundPeak3)
        areas_arr = np.array([[area_peak1], [area_peak2], [area_peak3]])
        emission_areas.append(areas_arr)
    emission_areas = np.concatenate(emission_areas, axis = 1).T
    return emission_areas
## calculate relative PLQY
def rel_PLQY(ABS_sample, areas_sample):
    # data related to the dye and solvent
    PLQY_dye = 0.675
    ABS_dye = 0.15781
#     area1_dye = 1447050
    area2_dye = 205117.25 * 3
    area3_dye = 16094.91 * 2.2
    etha_dye = 1.337
    etha_ODE = 1.444
    # calculate relative_PLQY
#     PLQY1 = PLQY_dye * (areas_sample[:, [0]] * ABS_dye * etha_ODE) / (area1_dye * ABS_sample * etha_dye)
    PLQY2 = PLQY_dye * (areas_sample[:, [0]] * ABS_dye * etha_ODE) / (area2_dye * ABS_sample * etha_dye)
    PLQY3 = PLQY_dye * (areas_sample[:, [1]] * ABS_dye * etha_ODE) / (area3_dye * ABS_sample * etha_dye)
    PLQY_2_3 = PLQY2 + PLQY3
#     relative_PLQY = PLQY1 + PLQY2 + PLQY3
    # return relative PLQY
    return PLQY_2_3

### ML Modeling

In [ ]:
## scale output
def non_dim_y(y_initial):
    y_scaled = np.zeros((y_initial.shape[0], y_initial.shape[1]))
    y_scaled[:, 0] = (y_initial[:, 0]) / (2 * 0.25)
    y_scaled[:, 1] = (y_initial[:, 1]) / (2 * 360476)
    y_scaled[:, 2] = (y_initial[:, 2]) / (2 * 71768)
    return y_scaled
## scale output back
def dim_y(y_initial):
    y_scaled = np.zeros((y_initial.shape[0], y_initial.shape[1]))
    y_scaled[:, 0] = (y_initial[:, 0]) * (2 * 0.25)
    y_scaled[:, 1] = (y_initial[:, 1]) * (2 * 360476)
    y_scaled[:, 2] = (y_initial[:, 2]) * (2 * 71768)
    return y_scaled
## nondimensionalize inputs
def non_dim_x(x_initial):
    x_normalized = np.zeros((x_initial.shape[0], x_initial.shape[1] - 3))
    x_normalized[:, 0] = (x_initial[:, 0] - 160) / (255 - 160) # x1 = temperature
    x_normalized[:, 1] = (x_initial[:, 1] - 45) / (50 - 45) # x2 = ODE
    x_normalized[:, 2] = (x_initial[:, 2] - 10) / (30 - 10) # x3 = OA
    x_normalized[:, 3] = (x_initial[:, 3] - 10) / (30 - 10) # x4 = OAm
    x_normalized[:, 4] = (x_initial[:, 4] - 20) / (70 - 20) # x5 = Cl
    x_normalized[:, 5] = (x_initial[:, 5] - 25) / (80 - 25) # x6 = Yb
    x_normalized[:, 6] = (x_initial[:, 6] - 10) / (40 - 10) # x7 = Mn
    x_normalized[:, 7] = (x_initial[:, 7] - 15) / (50 - 15) # x8 = Pb
    x_normalized[:, 8] = (x_initial[:, 8] - 15) / (50 - 15) # x8 = Cs
    return x_normalized
## split dataset to training and validation
def split_set(x, y, train_ratio = 0.8, seed_num = 100):
    xy_all = np.concatenate((x, y), axis = 1)
    np.random.seed(seed_num)  
    np.random.shuffle(xy_all)
    # training set
    xy_train = xy_all[:int(train_ratio * xy_all.shape[0]), :]
    x_train = xy_train[:, :-y.shape[1]]
    y_train = xy_train[:, -y.shape[1]:]
    # validation set
    xy_test = xy_all[int(train_ratio * xy_all.shape[0]):, :]
    x_test = xy_test[:, :-y.shape[1]]
    y_test = xy_test[:, -y.shape[1]:]
    return x_train, y_train, x_test, y_test
## ENN structure
def structure_info(num_models, layer_min = 8, layer_max = 10, node_min = 15, node_max = 30, seed_num = 100):
    random.seed(seed_num)
    detail_all = []
    for i in range (num_models):
        num_layers = random.randint(layer_min, layer_max)
        nodes_list = []
        for j in range(num_layers):
            num_nodes = random.randint(node_min, node_max)
            nodes_list.append(num_nodes)
        detail = [num_layers, nodes_list]
        detail_all.append(detail)
    return detail_all
## build and train an ensemble of cascade NNs
def parallel_train_ML(train_x, train_y, val_x, val_y, structure):
#     p = psutil.Process()
#     p.nice(psutil.REALTIME_PRIORITY_CLASS)
    ## cascade NN
    input = Input(shape = train_x.shape[1])
    x = input
    for layer in range(structure[0]):
        output = Dense(structure[1][layer], activation = 'relu', kernel_initializer = HeUniform(seed = 42), bias_initializer = Zeros())(x)
        x = Concatenate()([x, output])
    # dropout = Dropout(0.1)(x)
    # output = Dense(train_y.shape[1], activation = 'linear')(dropout)
    output = Dense(train_y.shape[1], activation = 'linear')(x)
    model = Model(inputs = input, outputs = output)
    ## train the model
    # early stopping terminates training if validation model loss begins to increase due to over-fitting
    # early_stopping = EarlyStopping(monitor = 'val_loss', patience = 50, min_delta = 1e-5, mode = 'min', restore_best_weights = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 50, min_delta = 1e-5, mode = 'min')
    # model compile with loss, optimizer, and performance metric of interest
    model.compile(loss = 'mean_squared_error', optimizer = RMSprop(learning_rate = 1e-4), metrics = [RootMeanSquaredError()])
    # perform model training with training and validation data with early stopping
    history = model.fit(train_x, train_y, validation_data = (val_x, val_y), epochs = 1000, verbose = 0, callbacks = [early_stopping])
    result = [model, history]
    return result
## deconvolute the model and training history
def deconvolute_ML(ensemble_result):
    ensemble_model = []
    ensemble_history = []
    for element in ensemble_result:
        ensemble_model.append(element[0])
        ensemble_history.append(element[1])
    return ensemble_model, ensemble_history
## plots related to the training process
def learn_plot(history):
    fig = plt.figure()
    # loss values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], color = 'blue')
    plt.plot(history.history['val_loss'], color = 'red')
    plt.title('Loss (MSE) - Epoch')
    plt.ylabel('Loss (MSE)')
    plt.xlabel('Epoch Number')
    plt.legend(['Training', 'Validation'])
    # metric values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['root_mean_squared_error'], color = 'blue')
    plt.plot(history.history['val_root_mean_squared_error'], color = 'red')
    plt.title('RMSE - Epoch')
    plt.ylabel('RMSE')
    plt.xlabel('Epoch Number')
    plt.legend(['Training', 'Validation'])
    fig.tight_layout()
    plt.show()
## plot related to the predicted vs. true outputs - for individual models   
def predict_plot_separate(model, x, y_true):
    predicted_y = model.predict(x, verbose = 0)
    detail_1 = linregress(y_true[:, 0], predicted_y[:, 0])
    detail_2 = linregress(y_true[:, 1], predicted_y[:, 1])
    detail_3 = linregress(y_true[:, 2], predicted_y[:, 2])
#     detail_4 = linregress(y_true[:, 3], predicted_y[:, 3])
    plt.scatter(y_true[:, 0], predicted_y[:, 0], color = 'blue')
    plt.scatter(y_true[:, 1], predicted_y[:, 1], color = 'red')
    plt.scatter(y_true[:, 2], predicted_y[:, 2], color = 'cyan')
#     plt.scatter(y_true[:, 3], predicted_y[:, 3], color = 'yellow')
    plt.plot([0, 1],[0, 1], color = 'green')
    plt.ylabel('Predicted')
    plt.xlabel('Actual')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    # plt.legend(['y0: $R^2 = $' + str(detail_1.rvalue ** 2), 'y1: $R^2 = $' + str(detail_2.rvalue ** 2), 'y2: $R^2 = $' + str(detail_3.rvalue ** 2), 'y3: $R^2 = $' + str(detail_4.rvalue ** 2), 'Parity'])
    plt.legend(['y0: $R^2 = $' + str(detail_1.rvalue ** 2), 'y1: $R^2 = $' + str(detail_2.rvalue ** 2), 'y2: $R^2 = $' + str(detail_3.rvalue ** 2), 'Parity'])
    plt.show()
## plot related to the predicted vs. true outputs - for the entire ENN   
def predict_plot_mean(ensemble_NN, x, y_true):
    y_pred = []
    for model in ensemble_NN:
        y_pred.append(model.predict(x, verbose = 0))
    y_pred_arr = np.concatenate(y_pred, axis = 1)
    dim = y_true.shape[1]
    y_pred_mean = []
    y_pred_std = []
    plt.figure(figsize = (3.5 * dim, 3.5), dpi = 200)
    for i in range(dim):
        element = y_pred_arr[:, i::dim]
        element_mean = np.mean(element, axis = 1)
        element_std = np.std(element, axis = 1)
        element_detail = linregress(y_true[:, i], element_mean)
        y_pred_mean.append(element_mean[:, np.newaxis])
        y_pred_std.append(element_std[:, np.newaxis])
        plt.subplot(1, dim, i + 1)
        plt.errorbar(y_true[:, i], element_mean, yerr = element_std, fmt = 'o', color = 'blue', alpha = 0.8, markersize = 5, label = 'y' + str(i) + ': $R^2 = $' + str(round((element_detail.rvalue ** 2), 4)))
        plt.plot([0, 1],[0, 1], color = 'red', label = 'Parity')
        # plt.title('y' + str(i + 1))
        plt.ylabel('Predicted')
        plt.xlabel('Actual')
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.legend()
    plt.tight_layout()
    pred_mean = np.concatenate(y_pred_mean, axis = 1)
    pred_std = np.concatenate(y_pred_std, axis = 1)
    return pred_mean, pred_std
## save the ENN in the given path
def save_ENN(ENN, path):
    for n, model in enumerate(ENN):
        os.mkdir(path + "//" + str(n))
        model.save(path + "//" + str(n))
## load the ENN stored in the given path
def load_plot(path):
    model_idx = sorted(os.listdir(path), key = len)
    ensemble = []
    for idx in model_idx:
        ensemble.append(load_model(path + "//" + idx))
    return ensemble

### Optimization

In [ ]:
# used of expected improvement (EI)
def best_target_data(y_norm, y_target):
    obj_fun_data = abs(y_norm - y_target)
    obj_fun_best = obj_fun_data.min()
    return obj_fun_best
## aquisition functions implementation
def apply_policy(x_initial, ensemble, y_target, obj_fun_best, policy):
    # p = psutil.Process()
    # p.nice(psutil.REALTIME_PRIORITY_CLASS)    
    x = x_initial[np.newaxis, :]
    obj_models = []
    for model in ensemble:
        y_pred = model.predict(x, verbose = 0)
        y_pred_dim = dim_y(y_pred)
        combination_pred = rel_PLQY(y_pred_dim[:, [0]], y_pred_dim[:, 1:]) / 2
        # our ptoblem is single objective optimization
        obj_fun = abs(y_target - np.squeeze(combination_pred)[()])
        obj_models.append(obj_fun)
    ## EI
    if (policy[0] == "EI"):
        imp_list = []
        for element in obj_models:
            diff = obj_fun_best - element 
            if (diff >= 0):
                imp_list.append(diff)
            else:
                imp_list.append(0)
        value = - np.mean(imp_list)
    ## UCB
    if (policy[0] == "UCB"):
        value = np.mean(obj_models) - (policy[1] * np.std(obj_models))
    ## MV      
    if (policy[0] == "MV"):
        value = - np.std(obj_models)
    ## EPLT
    elif (policy[0] == "EPLT"):
        value =  np.mean(obj_models)
    return value
## optimization
def opt_general(f, dim):
    # p = psutil.Process()
    # p.nice(psutil.REALTIME_PRIORITY_CLASS)
    low_limit = [0] * dim
    up_limit = [1] * dim
    bnds = list(zip(low_limit, up_limit))
    count_success = 0
    while (count_success < 1):
        x_initial = np.random.uniform(0, 1, dim)
        # sol = minimize(f, x_initial, method = "trust-constr", bounds = bnds, options = {'gtol': 0.02, 'xtol': 0.02, 'verbose': 0})
        sol = minimize(f, x_initial, method = "SLSQP", bounds = bnds, options = {'ftol': 0.02, 'maxiter': 1000})
        if ((sol.success == True) and (sol.nit > 1)):
            result = sol
            count_success = count_success + 1 
    return result      
def parallel_opt(ensemble, y_target, obj_fun_best, policy, x_shape):
#     p = psutil.Process()
#     p.nice(psutil.REALTIME_PRIORITY_CLASS)
    model_pol = lambda x: apply_policy(x, ensemble, y_target, obj_fun_best, policy)
    solution = opt_general(model_pol, x_shape)
    return solution
## deconvolution function while running only one optimization
def deconvolute_opt(solution, dim, num_restart, num_selection):
    xy = np.zeros((num_restart, dim + 1))
    for count in range(len(solution)):
        xy[count, :-1] = solution[count].x
        xy[count, -1] = solution[count].fun
    select_xy = xy[:num_selection, :]
    return select_xy
## function for experiment selection while running only one optimization
def exp_sel(y, FR, y_target, policy, num_selection, num_models = 20, num_restart = 1):
    ## dataset and preprocessing needed to be able to train the model
    y_norm = non_dim_y(y)
    combination_true = rel_PLQY(y[:, [0]], y[:, 1:]) / 2.0
    obj_fun_best = best_target_data(combination_true, y_target)
    # a = y.shape[0]
    # x_norm = non_dim_x(FR[:a, :])
    x_norm = non_dim_x(FR)
    x_train, y_train, x_valid, y_valid = split_set(x_norm, y_norm)
    ## train the ENN
    start_ML = time()
#     p = psutil.Process()
#     p.nice(psutil.REALTIME_PRIORITY_CLASS)
    arthitecture = structure_info(num_models)
    result_aggregate_ML = Parallel(n_jobs = -2)(delayed(parallel_train_ML)(x_train, y_train, x_valid, y_valid, arthitecture[i]) for i in range(num_models))
    ensemble, history = deconvolute_ML(result_aggregate_ML)
    end_ML = time()
    print("------------------------------------------------------------------------------------------------------------")
    print(f"The execution time to learn the ensemble NN is {str(end_ML - start_ML)} seconds.")
    print("------------------------------------------------------------------------------------------------------------")
    ## optimization and achive the best input variables needed for the future experiments
    start_opt = time()
#     p = psutil.Process()
#     p.nice(psutil.REALTIME_PRIORITY_CLASS)
    result_aggregate_opt = Parallel(n_jobs = -2)(delayed(parallel_opt)(ensemble, y_target, obj_fun_best, policy, x_norm.shape[1]) for i in range(num_restart))
    xy_best = deconvolute_opt(result_aggregate_opt, x_norm.shape[1], num_restart, num_selection)
    # sort based on the temperature
    x_best = xy_best[:, :-1]
    y_best = xy_best[:, [-1]]
    end_opt = time()
    print("------------------------------------------------------------------------------------------------------------")
    print(f"The execution time for optimization is {str(end_opt - start_opt)} seconds.")
    print("------------------------------------------------------------------------------------------------------------")
    return x_best, y_best, result_aggregate_opt, ensemble, history, x_train, y_train, x_valid, y_valid, arthitecture

### Master Code

In [ ]:
def count_files(path):
    files_all = os.listdir(path)
    count_ABS = 0
    count_PL = 0
    for file in files_all:
        if (file.startswith("Abs")):
            count_ABS = count_ABS + 1
        elif (file.startswith("PL")):
            count_PL = count_PL + 1
    return count_ABS, count_PL
def x_dim(x_norm, FR, path):
    x_final = np.zeros((x_norm.shape[0], x_norm.shape[1] + 3))
    x_final[:, 0] = (x_norm[:, 0] * (255 - 160)) + 160
    x_final[:, 1] = (x_norm[:, 1] * (50 - 45)) + 45
    x_final[:, 2] = (x_norm[:, 2] * (30 - 10)) + 10
    x_final[:, 3] = (x_norm[:, 3] * (30 - 10)) + 10
    x_final[:, 4] = (x_norm[:, 4] * (70 - 20)) + 20
    x_final[:, 5] = (x_norm[:, 5] * (80 - 25)) + 25
    x_final[:, 6] = (x_norm[:, 6] * (40 - 10)) + 10
    x_final[:, 7] = (x_norm[:, 7] * (50 - 15)) + 15
    x_final[:, 8] = (x_norm[:, 8] * (50 - 15)) + 15
    # third and second last columns
    x_final[:, -3] = (1.2 * np.sum(x_final[:, 1:-3], axis = 1) * 7.46) / 1000 # x10 = MFC
    x_final[:, -2] = 3 # x11 = 3 always
    # last column is still 0 but after concatenating, we will apply the washing after every 6 rows
    updated_FR = np.concatenate((FR, x_final), axis = 0) ## concat previous and new input parameters
    # last column: 1 in the rows divisible by 7 for washing otherwise 0
    for i in range(updated_FR.shape[0]):
        if ((i + 1) % 7 == 0):
            updated_FR[i, -1] = 1
    np.savetxt(path + "//" + "FR.csv", updated_FR, delimiter = ",", fmt="%1.3f") # overwrite it into the existing FR.csv file
    return x_final, updated_FR
## master function
## policy can be ["MV", 0], ["EPLT", 0], ["EI", 0], or ["UCB", 1. / math.sqrt(2)]
# def master_code(y_target = 1., policy = ["MV", 0], num_selection = 1, exp_budget = 160): # if MV is used
# def master_code(y_target = 1., policy = ["EPLT", 0], num_selection = 1, exp_budget = 160): # if EPLT is used
# def master_code(y_target = 1., policy = ["EI", 0], num_selection = 1, exp_budget = 160): # if EI is used
# def master_code(y_target = 1., policy = ["EI", 0], num_selection = 1, exp_budget = 13): # if EI without prior knowledge is used; initiated with 3 random experimental condition
def master_code(y_target = 1., policy = ["UCB", 1. / math.sqrt(2)], num_selection = 1, exp_budget = 160): # if UCB is used
    loc = input("Please enter the path for the directory that includes files: ")
    print("------------------------------------------------------------------------------------------------------------")
    num_files_old = 0
    while True:
        num_ABS, num_PL = count_files(loc)
        if (num_ABS == num_PL):
            num_files = num_PL
        if (num_files > num_files_old + num_selection - 1) and (num_files < exp_budget):
            print("New run starts now:")
            num_files_old = num_files
            WL_ABS, WL_PL, DR_ABS, DR_PL, LR, ABS_all, PL_all, ABS_names, PL_names, FR = read_files(loc)
            y_abs = spectra_extract_ABS(WL_ABS, DR_ABS, LR, ABS_all, ABS_names)
            y_areas = spectra_extract_PL(WL_PL, DR_PL, PL_all, PL_names)
            y = np.concatenate((y_abs, y_areas[:, 1:]), axis = 1)
            # y1, y2, y3, y = rel_PLQY(y_abs, y_areas)
            # y_final = y2 + y3
            new_x, new_y, solution_all, ensemble, history, x_train, y_train, x_valid, y_valid, structure = exp_sel(y, FR, y_target, policy, num_selection)
            new_FR, updated_FR = x_dim(new_x, FR, loc)
        elif (num_files >= exp_budget):
            print("The experimental budget is reached!")
            break       
#     return new_x, new_y, solution_all, ensemble, history, x_train, y_train, x_valid, y_valid, structure, new_FR, updated_FR

In [ ]:
# new_x, new_y, solution_all, ensemble, history, x_train, y_train, x_valid, y_valid, structure, new_FR, updated_FR = master_code()
master_code()